In [0]:
%pip install fiona py7zr tqdm shapely pyproj geopandas

In [0]:
catalog = "lucasbruand_catalog"
schema = "ign_bdtopo"
volume = "bronze_volume"
file_prefix = "fileprefix"
dept="D001"
layer = "batiment"
target_table = f"ign_bdtopo_{layer}"

In [0]:
orig_file = "/Volumes/lucasbruand_catalog/ign_bdtopo/bronze_volume/fileprefix/BDTOPO_3-5_TOUSTHEMES_GPKG_LAMB93_D001_2025-09-15.7z"
subfile_in_archive = ""

In [0]:
import py7zr

def list_files_in_7z(archive_path):
    """
    List all files contained in a .7z archive using py7zr.
    Args:
        archive_path (str): Path to the .7z archive file.
    Returns:
        List[str]: List of file names inside the archive.
    """
    with py7zr.SevenZipFile(archive_path, mode='r') as archive:
        return archive.getnames()


In [0]:
def get_payload_filename(orig_file):
  return [ filename for filename in list_files_in_7z(orig_file) if filename.endswith(".gpkg")][0]

In [0]:
table_schema = """
Table schema:
root
-- id: string (nullable = true)
-- dept: string (nullable = true)
-- layer: string (nullable = true)
-- cleabs: string (nullable = true)
-- nature: string (nullable = true)
-- usage_1: string (nullable = true)
-- usage_2: string (nullable = true)
-- construction_legere: boolean (nullable = true)
-- etat_de_l_objet: string (nullable = true)
-- date_creation: timestamp (nullable = true)
-- date_modification: timestamp (nullable = true)
-- date_d_apparition: timestamp (nullable = true)
-- date_de_confirmation: timestamp (nullable = true)
-- sources: string (nullable = true)
-- identifiants_sources: string (nullable = true)
-- methode_d_acquisition_planimetrique: string (nullable = true)
-- methode_d_acquisition_altimetrique: string (nullable = true)
-- precision_planimetrique: double (nullable = true)
-- precision_altimetrique: double (nullable = true)
-- nombre_de_logements: double (nullable = true)
-- nombre_d_etages: double (nullable = true)
-- materiaux_des_murs: string (nullable = true)
-- materiaux_de_la_toiture: string (nullable = true)
-- hauteur: double (nullable = true)
-- altitude_minimale_sol: double (nullable = true)
-- altitude_minimale_toit: double (nullable = true)
-- altitude_maximale_toit: double (nullable = true)
-- altitude_maximale_sol: double (nullable = true)
-- origine_du_batiment: string (nullable = true)
-- appariement_fichiers_fonciers: string (nullable = true)
-- identifiants_rnb: string (nullable = true)
-- geometry: string (nullable = true)"""



In [0]:
spark.sql(f"DROP TABLE IF EXISTS {catalog}.{schema}.{target_table}")

In [0]:
import re

def parse_table_schema(schema_str, table_name):
    lines = schema_str.strip().split('\n')
    columns = []
    for line in lines:
        m = re.match(r"-- ([\w_']+): ([\w\s\(\)]+)", line.strip())
        if m:
            col = m.group(1).replace("'", "")
            typ = m.group(2).split()[0]
            # Map Spark types to SQL types
            type_map = {
                "string": "STRING",
                "boolean": "BOOLEAN",
                "double": "DOUBLE",
                "integer": "INT",
                "date": "DATE",
                "timestamp": "TIMESTAMP"
            }
            sql_type = type_map.get(typ, "STRING")
            if col == "geometry":
                #columns.append(f"`{col}` GEOMETRY (4326)")
                columns.append(f"`{col}` {sql_type}")
            else:
                columns.append(f"`{col}` {sql_type}")
    cols_str = ",\n  ".join(columns)
    return f"CREATE TABLE IF NOT EXISTS {table_name} (\n  {cols_str}\n) USING DELTA"

create_stmt = parse_table_schema(table_schema, f"{catalog}.{schema}.{target_table}")
print(create_stmt)

In [0]:
spark.sql(create_stmt)

In [0]:
import py7zr
import tempfile
import os
import geopandas as gpd

def load_gpkg_from_7z_to_geopandas(orig_file, dept, layer=None):
    """
    Extract the .gpkg file from a 7z archive and load up to 100 rows into a GeoPandas DataFrame.
    Args:
        orig_file (str): Path to the .7z archive.
        layer (str or None): Layer name to load from the .gpkg file (default: None, loads the first layer).
    Returns:
        geopandas.GeoDataFrame: The loaded geospatial data (up to 100 rows).
    """
    with py7zr.SevenZipFile(orig_file, mode='r') as archive:
        payload_files = [name for name in archive.getnames() if name.endswith('.gpkg')]
        if not payload_files:
            raise FileNotFoundError("No .gpkg file found in archive.")
        payload_name = payload_files[0]
        with tempfile.TemporaryDirectory() as tmpdir:
            archive.extract(targets=[payload_name], path=tmpdir)
            payload_path = os.path.join(tmpdir, payload_name)
            gdf = gpd.read_file(payload_path, layer=layer, rows=1000)
            gdf["dept"] = dept
            gdf["layer"] = layer
            return gdf


In [0]:
df = load_gpkg_from_7z_to_geopandas(orig_file, dept="D001", layer=layer)

In [0]:
df['geometry'] = df['geometry'].astype(str)
sdf = spark.createDataFrame(df)

In [0]:
full_table_name = f"{catalog}.{schema}.{target_table}"
sdf.write.format("delta").mode("append").saveAsTable(full_table_name)